<a href="https://colab.research.google.com/github/maverick98/Group4Capstone/blob/main/%5BFlickr8k_token_txt%5D_CNN_RNN_20APR2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Necessary Packages

In [ ]:
#!pip install keras
!pip install gensim
!pip install python-levenshtein

# Import necessary libraries

In [50]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector
from keras.layers import Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
#from tensorflow.keras.layers.wrappers import Bidirectional
#from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
#from tf.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np
from numpy import array
import pandas as pd
import cv2
from glob import glob
import PIL
import time
from tqdm import tqdm
import os
import gensim
import random
import time
from google.colab import drive
from os.path import join
from importlib.machinery import SourceFileLoader     


# Check if connected to GPU

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [49]:
!rm -rf /content/group4capstone-code

# Mount Google Drive

In [5]:


ROOT = "/content/drive"
drive.mount(ROOT,force_remount=True)




Mounted at /content/drive


# Download the flickr8k dataset and Glove Embeddings

In [ ]:
#Uncomment for the first time.
!wget  https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_Dataset.zip
!wget https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_text.zip
!wget https://nlp.stanford.edu/data/glove.6B.zip

In [43]:
PROJ = "MyDrive/Capstone/src" 
PROJECT_PATH = join(ROOT, PROJ)
print("PROJECT_PATH from your Google Drive is ",PROJECT_PATH)
!rm -rf "{PROJECT_PATH}"
!mkdir  "{PROJECT_PATH}"

PROJECT_PATH from your Google Drive is  /content/drive/MyDrive/Capstone/src


# Store it under dataset directory

In [51]:
CUR_DIR='/content'


In [52]:
!rm -rf {CUR_DIR}/dataset
!mkdir -p {CUR_DIR}/dataset/images
!mkdir -p {CUR_DIR}/dataset/texts
!mkdir -p {CUR_DIR}/dataset/glove

!unzip  {CUR_DIR}/Flickr8k_text.zip -d {CUR_DIR}/dataset/texts
!unzip {CUR_DIR}/Flickr8k_Dataset.zip -d {CUR_DIR}/dataset/images
!unzip  {CUR_DIR}/glove.6B.zip -d {CUR_DIR}/dataset/glove

# Clone the code from https://sourceforge.net/projects/group4capstone/

In [172]:
%cd "{PROJECT_PATH}"
!pwd
!rm -rf /content/drive/MyDrive/Capstone/src/group4capstone-code
MY_USER_NAME = 'msahu98' # This is your sourceforge.net username
!git clone https://{MY_USER_NAME}@git.code.sf.net/p/group4capstone/code group4capstone-code
%cd {CUR_DIR}

/content/drive/MyDrive/Capstone/src
/content/drive/MyDrive/Capstone/src
Cloning into 'group4capstone-code'...
remote: Enumerating objects: 1073, done.
remote: Counting objects: 100% (1073/1073), done.
remote: Compressing objects: 100% (875/875), done.
remote: Total 1073 (delta 550), reused 0 (delta 0)
Receiving objects: 100% (1073/1073), 156.52 KiB | 725.00 KiB/s, done.
Resolving deltas: 100% (550/550), done.
/content


# Load Modules

In [173]:
!pwd

/content


In [174]:
def load_module(module_name,module_path):
    module_py=module_name+'.py'
    SourceFileLoader(module_name, join(join(PROJECT_PATH,'group4capstone-code/src/',module_path), module_py)).load_module()



In [175]:
import pathlib,sys,os

sys.path.append(os.path.dirname(join(PROJECT_PATH,'group4capstone-code/')))

In [176]:
load_module('cnn_model_sequence','model/data')
load_module('dataset_input_sequential','model/data')
load_module('dataset_input_sequential_factory','model/data')
load_module('image_caption_dataset_sequential','model/data')
load_module('train_sequential','train')



In [177]:
!pip3 freeze

absl-py==1.4.0
alabaster==0.7.13
albumentations==1.2.1
altair==4.2.2
anyio==3.6.2
appdirs==1.4.4
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arviz==0.15.1
astropy==5.2.2
astunparse==1.6.3
attrs==23.1.0
audioread==3.0.0
autograd==1.5
Babel==2.12.1
backcall==0.2.0
beautifulsoup4==4.11.2
bleach==6.0.0
blis==0.7.9
blosc2==2.0.0
bokeh==2.4.3
branca==0.6.0
CacheControl==0.12.11
cached-property==1.5.2
cachetools==5.3.0
catalogue==2.0.8
certifi==2022.12.7
cffi==1.15.1
chardet==4.0.0
charset-normalizer==2.0.12
chex==0.1.7
click==8.1.3
cloudpickle==2.2.1
cmake==3.25.2
cmdstanpy==1.1.0
colorcet==3.0.1
colorlover==0.3.0
community==1.0.0b1
confection==0.0.4
cons==0.4.5
contextlib2==0.6.0.post1
contourpy==1.0.7
convertdate==2.4.0
cryptography==40.0.2
cufflinks==0.17.3
cvxopt==1.3.0
cvxpy==1.3.1
cycler==0.11.0
cymem==2.0.7
Cython==0.29.34
dask==2022.12.1
datascience==0.17.6
db-dtypes==1.1.1
dbus-python==1.2.16
debugpy==1.6.6
decorator==4.4.2
defusedxml==0.7.1
distributed==2022.12.1
dlib==19.24.1

# Import image caption libs

In [178]:
from cnn_model_sequence import CNN_Model
from dataset_input_sequential import DatasetInput
from dataset_input_sequential_factory import DatasetInputSequentialFactory
from image_caption_dataset_sequential import ImageCaptionDataset
from train_sequential import Train




# Initialize model parameters

In [ ]:
datasetInputFactory=DatasetInputSequentialFactory()
datasetInput=datasetInputFactory.create()
imageCaptionDataset= ImageCaptionDataset(datasetInput)
imageCaptionDataset.pretrain()
train_sequential=Train(imageCaptionDataset)
train_sequential.train()

Performing pretrain activities
Processing tokens
loading descriptions
Found descriptions of legnth  8092
Sanitizing descriptions
Building Vocabulary
Original Vocabulary Size: 6686
Saving descriptions to  /content/drive/MyDrive/Capstone/sequential/descriptions.txt
Train imgs Size: 6000
Test imgs Size: 1000
Descriptions: train=6000
preprocessed words 5761 -> 1342
Found 400000 word vectors.
Model: "model_14"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_19 (InputLayer)          [(None, 32)]         0           []                               
                                                                                                  
 input_18 (InputLayer)          [(None, 2048)]       0           []                               
                                                                                                